### Imports

In [15]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import re
import nltk
import logging

from sklearn import feature_extraction
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (plot_roc_curve, classification_report, roc_auc_score, average_precision_score,
confusion_matrix, plot_confusion_matrix, plot_precision_recall_curve)

### Configs

In [16]:
configs = {
    "run_name": "template_run",
    "test_year": 2020,
    # Model Type, will support LR, SVC, RandomForest, and NB
    'model': 'rf',
    # Vectorizer Type (CV, TFIDF)
    'vectorizer': {'type': 'cv', 
                    'ngrams': (1,3),
                   'binary': False,
                   'stopwords': 'english',
                   'max_df': 0.8,
                   'min_df': 10
                  },
    'grid_search': True, # If true, your other hyperparemeters should be in a list.
    'hyperparameters': {
        # Change these based on your model.
        "n_estimators": [70],
        "min_samples_split":[8]
        #'max_depth': [1, 2, 4, 8]
    }
}

configs

{'run_name': 'template_run',
 'test_year': 2020,
 'model': 'rf',
 'vectorizer': {'type': 'cv',
  'ngrams': (1, 3),
  'binary': False,
  'stopwords': 'english',
  'max_df': 0.8,
  'min_df': 10},
 'grid_search': True,
 'hyperparameters': {'n_estimators': [70], 'min_samples_split': [8]}}

In [17]:
ts = str(datetime.datetime.now())[:19].replace(":","-")
logging.basicConfig(filename = f"../logs/{configs['run_name']}_{ts}.log", level=logging.INFO)

In [18]:
logging.info(configs)

### Data Loading and Splitting

In [19]:
data = pd.read_csv('../data/combined_clean.csv', sep = "|")
print(data.shape)
logging.info(f"Date shape before removing duplicates {data.shape}")
data = data.drop_duplicates().reset_index(drop = True)
print(data.shape)
logging.info(f"Date shape after removing duplicates {data.shape}")


(215013, 4)
(211631, 4)


In [20]:
data['clean'] = data['clean'].astype('U')
data['year'] = data['year'].astype(int)

In [21]:
test_year = configs['test_year']

train = data.loc[data['year'] < test_year]
test = data.loc[data['year'] == test_year]

train_sarcastic_pct = train['is_sarcastic'].mean()*100
test_sarcastic_pct = test['is_sarcastic'].mean()*100

print(f"Train has {train.shape[0]} records, {round(train_sarcastic_pct,2)}% are sarcastic headlines")
print(f"Test has {test.shape[0]} records, {round(test_sarcastic_pct,2)}% are sarcastic headlines")

logging.info(f"Train has {train.shape[0]} records, {round(train_sarcastic_pct,2)}% are sarcastic headlines")
logging.info(f"Test has {test.shape[0]} records, {round(test_sarcastic_pct,2)}% are sarcastic headlines")

Train has 177565 records, 10.03% are sarcastic headlines
Test has 34066 records, 14.56% are sarcastic headlines


In [22]:
X_train = train['clean']
y_train = train['is_sarcastic']

X_test = test['clean']
y_test = test['is_sarcastic']

### Vectorization

Please adjust the configs or add other data processing steps here.

In [23]:
print("Please make sure your vectorizer is a valid option or add the functionality.")
print('Vectorizer is what converts the text into numerical entries')

if configs['vectorizer']['type'] == "cv":
    vectorizer = feature_extraction.text.CountVectorizer(
                                             lowercase = True,
                                             strip_accents = 'ascii',
                                             stop_words = configs['vectorizer']['stopwords'],
                                             ngram_range = configs['vectorizer']['ngrams'],
                                             max_df = configs['vectorizer']['max_df'],
                                             min_df = configs['vectorizer']['min_df'],
                                             binary = configs['vectorizer']['binary']
    )
                                                        
if configs['vectorizer']['type'] == "tfidf":
    vectorizer = feature_extraction.text.TfidfVectorizer(
                                             lowercase = True,
                                             strip_accents = 'ascii',
                                             stop_words = configs['vectorizer']['stopwords'],
                                             ngram_range = configs['vectorizer']['ngrams'],
                                             max_df = configs['vectorizer']['max_df'],
                                             min_df = configs['vectorizer']['min_df'],
                                             binary = configs['vectorizer']['binary']
    )
    
assert(vectorizer)
vectorizer

logging.info(vectorizer)
print(vectorizer)

Please make sure your vectorizer is a valid option or add the functionality.
Vectorizer is what converts the text into numerical entries
CountVectorizer(max_df=0.8, min_df=10, ngram_range=(1, 3), stop_words='english',
                strip_accents='ascii')


In [24]:
%%time
print("Fitting Vectorizer and Transform Train & Test")
X_train_transformed =  vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

Fitting Vectorizer and Transform Train & Test
CPU times: user 6.11 s, sys: 377 ms, total: 6.49 s
Wall time: 6.98 s


### Modeling

In [25]:
if configs['grid_search']:
    if configs['model'] == "lr":
        model = LogisticRegression()
    if configs['model'] == "rf":
        model = RandomForestClassifier()
    if configs['model'] == "bnb":
        model = BernoulliNB()
    if configs['model'] == "mnb":
        model = MultinomialNB()
    if configs['model'] == 'svc':
        model = SVC()

    grid_search = GridSearchCV(model, configs['hyperparameters'])
    
logging.info(grid_search)
grid_search

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'min_samples_split': [8], 'n_estimators': [70]})

In [ ]:
grid_search.fit(X_train_transformed, y_train)

In [ ]:
be = grid_search.best_estimator_
logging.info("Best Model:")
logging.info(be)
be

### Evaluation

In [ ]:
be_preds = be.predict(X_test_transformed)
be_probs = be.predict_proba(X_test_transformed)[:,1]

print(classification_report(y_test, be_preds))

plot_confusion_matrix(be, X_test_transformed, y_test, cmap = plt.cm.Blues)
plt.show()

plot_roc_curve(be, X_test_transformed, y_test)
plt.show()

plot_precision_recall_curve(be, X_test_transformed, y_test)
plt.show()

## Save Information

In [15]:
logging.info(grid_search.cv_results_)

In [16]:
auc_score = roc_auc_score(y_test, be_probs)
avg_precision = average_precision_score(y_test, be_probs)
cm = confusion_matrix(y_test, be_preds)

In [17]:
logging.info(f'AUC: {auc_score}')
logging.info(f'Average Precision: {avg_precision}')
logging.info('Confusion Matrix:')
logging.info(cm)